In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
from PIL import Image
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
classes = os.listdir('E:\\Plant Doc Disease Classification\\PlantDoc-Dataset-master\\train')
tclasses = os.listdir('E:\\Plant Doc Disease Classification\\PlantDoc-Dataset-master\\test')
print(tclasses)

['Apple leaf', 'Apple rust leaf', 'Apple Scab Leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'grape leaf', 'grape leaf black rot', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato Septoria leaf spot', 'Tomato two spotted spider mites leaf']


In [3]:
datagen = ImageDataGenerator(rescale=1./255)
gray_train_datagen = datagen.flow_from_directory('E:\\Plant Doc Disease Classification\\PlantDoc-Dataset-master\\gray_train',
                                            target_size = (128,128),
                                            batch_size = 64,
                                            class_mode = 'categorical')
gray_test_datagen = datagen.flow_from_directory('E:\\Plant Doc Disease Classification\\PlantDoc-Dataset-master\\gray_test',
                                            target_size = (128,128),
                                            batch_size = 64,
                                            class_mode = 'categorical')
train_datagen = datagen.flow_from_directory('E:\\Plant Doc Disease Classification\\PlantDoc-Dataset-master\\train',
                                            target_size = (128,128),
                                            batch_size = 64,
                                            class_mode = 'categorical')
test_datagen = datagen.flow_from_directory('E:\\Plant Doc Disease Classification\\PlantDoc-Dataset-master\\test',
                                            target_size = (128,128),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 1960 images belonging to 28 classes.
Found 227 images belonging to 28 classes.
Found 1960 images belonging to 28 classes.
Found 227 images belonging to 28 classes.


In [9]:
rgb = tf.keras.layers.Input(shape=(128,128,3))
lab = tf.keras.layers.Input(shape=(128,128,3))

model1 = tf.keras.layers.Conv2D(32,kernel_size= (3,3), strides= (1,1), activation = 'relu')(rgb)
model1 = tf.keras.layers.AveragePooling2D((2,2))(model1)
model1 = tf.keras.layers.Conv2D(64,kernel_size= (3,3), strides= (1,1), activation = 'relu')(model1)
model1 = tf.keras.layers.AveragePooling2D((2,2))(model1)
out1 = tf.keras.layers.Flatten()(model1)
#model1 = tf.keras.Model(inputs = rgb, outputs = model1)

model2 = tf.keras.layers.Conv2D(32,kernel_size= (3,3), strides= (1,1), activation = 'relu')(lab)
model2 = tf.keras.layers.AveragePooling2D((2,2))(model2)
model2 = tf.keras.layers.Conv2D(64,kernel_size= (3,3), strides= (1,1), activation = 'relu')(model2)
model2 = tf.keras.layers.AveragePooling2D((2,2))(model2)
out2 = tf.keras.layers.Flatten()(model2)
#model2 = tf.keras.Model(inputs = lab, outputs = model2)

conc = tf.keras.layers.Concatenate()([out1, out2])

final = tf.keras.layers.Dense(128, activation= 'relu')(conc)
final = tf.keras.layers.Dense(len(classes), activation= 'softmax')(final)

model = tf.keras.Model(inputs = [rgb, lab], outputs = final)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_12 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 126, 126, 32  896         ['input_11[0][0]']               
                                )                                                             

In [13]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [10]:
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = ['Accuracy'])

In [ ]:
labels = tf.keras.utils.to_categorical()

In [11]:
model.fit([train_datagen, gray_train_datagen], epochs= 30, validation_data= [test_datagen, gray_test_datagen], verbose=1)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'keras.preprocessing.image.DirectoryIterator'>"}), <class 'NoneType'>